In [14]:
from chase.base import *
from chase.utils import *
from chase.fit import *
from mypy.explib.frey2014 import frey2014
pd.set_option('display.max_colwidth', 100)

In [4]:
data = pd.read_csv('frey_data.csv', index_col=0)
problems = {gid: frey2014.get_options(gid) for gid in data.problem.unique()}
data.head(10)

,subject,problem,group,choice,samplesize
0,1,S1G1,0,0,12
1,1,S2G1,0,1,10
2,1,S3G1,0,0,10
3,1,S4G1,0,0,10
4,1,S5G1,0,0,10
5,1,S6G1,0,1,10
6,1,S7G1,0,0,10
7,1,S8G1,0,0,12
8,1,S9G1,0,0,10
9,1,S10G1,0,0,10


# Fit individual subjects

In [40]:
emp_p_switch = {}
for sid in frey2014.subjects():
    arr = []
    for a in frey2014.streak_lengths_by_subject(sid).values:
        for x in a:
            arr.append(x)
    emp_p_switch[sid] = np.round(1 / np.mean(arr), 3)

In [38]:
# a list of possible free parameters and their ranges
# and starting point (optional)
PARS = {'theta': [1, 50],
        'p_stay': [0, 1],
        'tau': [0, 40, 1.],
        'prelec_gamma': [0, 5],
        'prelec_elevation': [0, 5],
        'pow_gain': [0., np.inf, 1.],
        'w_loss': [0., np.inf, 1.],
        'mu': [0., np.inf, 10.],
        'sc': [0., np.inf, 1.]}

# parameters that are set to fixed values 
FIXED = {'delta': 0.5,
         'theta': 30}

N_ITER = 3
OUTDIR = 'chase_fitresults_frey_individual'

# a list of different parameter combinations that will
# be fit
PARSETS = [['mu', 'sc', 'p_stay', 'tau'],
           ['mu', 'sc', 'p_stay', 'tau', 'prelec_gamma'],
           ['mu', 'sc', 'p_stay', 'tau', 'pow_gain'],
           ]

In [ ]:
sset = []

In [41]:
from chase.base import *
from chase.utils import *
from chase.fit import *


#for sid in data.subject.unique()[:1]:
for sid in [2]:

    FIXED['p_switch'] = emp_p_switch[sid]
    
    SIM_ID = 'frey_individual_planned_switching_subj=%s' % sid

    for parset in PARSETS:

        fitting = {p: PARS[p] for p in parset}

        # initialize the model
        m = CHASEAlternateStoppingSwitchingModel(drift='cpt',
                                                 startdist='softmax',
                                                 stoprule='truncatednormal')

        # fit
        results = fit_mlh(m, problems, data[data.subject==sid], 
                          SIM_ID, FIXED, fitting, niter=N_ITER, outdir=OUTDIR)

        print results.sort('nllh').head(1)

frey_individual_planned_switching_subj=2(mu,p_stay,sc,tau|delta=0.5,p_switch=0.136,theta=30)
0/3
theta: 30.0
1406.17158316
1418.85413534
1406.51848592
1378.80024652
1406.14481285
1383.80155134
1380.51346187
1364.574032
1331.22079583
1316.10363967
1255.85181129
1304.24929573
1257.85610976
1172.58383899
1082.94862908
1112.04520813
1065.55783536
1006.36996848
1008.92005743
959.520615391
939.98009924
961.723441068
964.364951786
967.292217035
997.637122271
974.42701506
957.611067799
975.478485885
957.273900752
954.454927232
933.150906073
917.89062426
926.481250834
920.397667381
902.33291175
888.413651452
878.806625937
850.659045562
857.954723262
851.055508965
808.188350759
765.783836632
772.00693154
772.287891736
724.287904674
672.160957842
658.968800616
601.432471514
631.228656791
704.15050361
567.950481981
655.084122713
557.769118009
621.370020023
609.148288723
603.368823549
555.735532115
585.603957451
538.072433036
568.827269301
561.212655582
559.772597407
543.103818591
553.983147603
552

KeyboardInterrupt: 

In [63]:
allresults = pd.DataFrame(columns=['sim_id', 'k', 'N', 'nllh', 'bic'])
for parset in PARSETS:
    fitting = {p: PARS[p] for p in parset}
    allresults.loc[allresults.shape[0]] = best_result('frey_individual_subj=%s' % sid, FIXED, fitting, outdir=OUTDIR, nopars=True)

allresults.sort('bic')

IOError: File chase_fitresults_frey_individual/frey_individual_subj=1(mu,p_stay,sc,tau|delta=0.5,theta=30).csv does not exist